In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy
import pandas
import torchvision 
import torchvision.transforms as transforms

In [2]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
training_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
val_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
training_loader=torch.utils.data.DataLoader(dataset=training_data,batch_size=4,shuffle=True)
val_loader=torch.utils.data.DataLoader(dataset=val_data,batch_size=4,shuffle=False)

In [4]:
class conv(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [5]:
model=conv()


In [6]:
loss_fn=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(params=model.parameters(),lr=0.01)

In [7]:
epochs=2
running_loss=0.0
for i in range(epochs):
  for i,data in enumerate(training_loader,0):
    input,labels=data
    output=model(input)
    optimiser.zero_grad()
    loss=loss_fn(output,labels)
    loss.backward()
    optimiser.step()
    running_loss+=loss.item()
    if i%2000==1999:
      print("The epoch number is {} and the loss is {}".format(i+1,running_loss/2000))
      running_loss=0.0

The epoch number is 2000 and the loss is 2.3066040258407594
The epoch number is 4000 and the loss is 2.306731802582741
The epoch number is 6000 and the loss is 2.30606123650074
The epoch number is 8000 and the loss is 2.3060640256404876
The epoch number is 10000 and the loss is 2.305259413719177
The epoch number is 12000 and the loss is 2.3064285974502563
The epoch number is 2000 and the loss is 2.8835079201459886
The epoch number is 4000 and the loss is 2.307024056315422
The epoch number is 6000 and the loss is 2.3066479403972626
The epoch number is 8000 and the loss is 2.3050088670253754
The epoch number is 10000 and the loss is 2.3070826858282087
The epoch number is 12000 and the loss is 2.306126936197281


In [8]:
total=0
correct=0
with torch.no_grad():
  for i,data in enumerate(val_loader,0):
    input,labels=data
    output=model(input)
    _,predicted=torch.max(output.data,1)
    correct+=(predicted==labels).sum().item()
    total+=labels.size(0)

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %
